In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:

import fnmatch
train = []
tes = []
val = []

for root,dirs,files in os.walk('./3fold_out4/train'):
    if files:
        lst = os.listdir(root)
        png = fnmatch.filter(lst,'*.png')
        if png:
            for i in png:
                train.append((root+'/'+i, int(i[3])-1))
for root,dirs,files in os.walk('./3fold_out4/test'):
      if files:
        lst = os.listdir(root)
        png = fnmatch.filter(lst,'*.png')
        if png:
            for i in png:
                tes.append((root+'/'+i, int(i[3])-1))
for root,dirs,files in os.walk('./3fold_out4/val'):
      if files:
        lst = os.listdir(root)
        png = fnmatch.filter(lst,'*.png')
        if png:
            for i in png:
                val.append((root+'/'+i, int(i[3])-1))

In [3]:
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import matplotlib.pyplot as plt

In [4]:
cout = [0]*4
for i in val:
    if i[1] == 0:
        cout[0]+=1
    if i[1] == 1:
        cout[1]+=1
    if i[1] == 2:
        cout[2]+=1
    if i[1] == 3:
        cout[3]+=1
print(cout,sum(cout))

[49, 20, 65, 44] 178


In [5]:
trn = train + tes
trn, _ = train_test_split(trn, test_size = 0.667)
test = val

In [6]:
class MyDataset(Dataset):
    def __init__(self, image_path, transform = None):
        self.image_list = image_path
        self.len = len(self.image_list)
        self.transform = transform
    def __getitem__(self, index):
        fn, label = self.image_list[index]
        img = Image.open(fn)
        label = np.array(label,dtype = np.float64)
        if self.transform is not None:
            img = self.transform(img) 
        return img, label
    def __len__(self):
        return self.len

Transform = transforms.Compose([
    transforms.RandomRotation(degrees=15),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    #transforms.CenterCrop(224),
    transforms.Resize(128),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ])
Lable_trasfrom = transforms.Compose([
    transforms.ToTensor()
    ])

In [7]:
train_data = MyDataset(trn, transform = Transform)
test_data = MyDataset(test, transform = Transform)
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=32, shuffle=False)

In [8]:
for i,(data,label) in enumerate(train_loader):
    print(data.shape)

torch.Size([32, 3, 128, 128])
torch.Size([32, 3, 128, 128])
torch.Size([32, 3, 128, 128])
torch.Size([7, 3, 128, 128])


In [9]:
import torchvision.models as models
from torch import nn,optim

In [10]:
my_resnet = models.resnet18(pretrained=True)

In [11]:
my_resnet.fc = nn.Sequential(
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.8),
    nn.Linear(128,4),
    nn.LogSoftmax(dim=1)
)
'''
my_resnet.classifier = nn.Sequential(
    nn.Linear(1024, 128),
    nn.ReLU(),
    nn.Dropout(0.6),
    nn.Linear(128,5),
    nn.LogSoftmax(dim=1)
)'''

'\nmy_resnet.classifier = nn.Sequential(\n    nn.Linear(1024, 128),\n    nn.ReLU(),\n    nn.Dropout(0.6),\n    nn.Linear(128,5),\n    nn.LogSoftmax(dim=1)\n)'

In [12]:
my_resnet = my_resnet.to('cuda:0')

In [13]:
#loss_func = FocalLoss()
#loss_func = NPairLoss()
loss_func = nn.NLLLoss()
lr = 1e-3
#optimizer = optim.Adam(my_resnet.parameters(), lr=lr)
#optimizer = optim.Adam(my_resnet.parameters(), lr=lr)
optimizer = optim.SGD(my_resnet.parameters(),
                                lr=lr,
                                momentum=0.9,
                                weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.1, last_epoch=-1)

In [14]:
import time

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
history = []
best_acc = 0.0
best_epoch = 0
best_each = ''

b = 0.3
best_loss = 100.0

In [16]:
for epoch in range(100):
    epoch_start = time.time()
    print("Epoch: {}/{}".format(epoch+1, 100))
    my_resnet.train()
    train_loss = 0.0
    valid_loss = 0.0
    train_acc = 0.0
    valid_acc = 0.0
    for i,(data,label) in enumerate(train_loader):
        data = data.to(device)
        label = label.to(device)
        label = label.long()
        optimizer.zero_grad()
        outputs = my_resnet(data)
        loss = loss_func(outputs,label)
        #loss = abs(loss-b) + b
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        ret, predictions = torch.max(outputs.data, 1)
        correct_counts = predictions.eq(label.data.view_as(predictions))
        acc = torch.mean(correct_counts.type(torch.FloatTensor))
        train_acc += acc.item() * data.size(0)
    with torch.no_grad():
        my_resnet.eval()
        correct = list(0. for i in range(4))
        total = list(0. for i in range(4))
        #correct = list(0. for i in range(2))
        #total = list(0. for i in range(2))
        for j, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels = labels.long()

            outputs = my_resnet(inputs)

            loss = loss_func(outputs, labels)
            #loss = abs(loss-b) + b
            valid_loss += loss.item() * inputs.size(0)
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            valid_acc += acc.item() * inputs.size(0)
            res = predictions == labels
            for label_idx in range(len(labels)):
                label_single = labels[label_idx]
                correct[label_single] += res[label_idx].item()
                total[label_single] += 1

    scheduler.step()
    avg_train_loss = train_loss/len(trn)
    avg_train_acc = train_acc/len(trn)

    avg_valid_loss = valid_loss/len(test)
    avg_valid_acc = valid_acc/len(test)

    acc_str = 'Accuracy: %f'%(sum(correct)/sum(total))

    for acc_idx in range(4):
    #for acc_idx in range(2):
        try:
            acc = correct[acc_idx]/total[acc_idx]
        except:
            acc = 0
        finally:
            acc_str += ' classID:%d acc:%f '%(acc_idx+1, acc)

    history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
    if best_acc <= avg_valid_acc:
        best_acc = avg_valid_acc
        best_epoch = epoch + 1
        best_each = acc_str

    epoch_end = time.time()

    print("Epoch: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation: Loss: {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch+1, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))
    print("Best Accuracy for validation : {:.4f} at epoch {:03d}".format(best_acc, best_epoch))
    print(acc_str)#print("Best Acc's each class acc:",best_each)
print("Best Acc's each class acc:",best_each)

Epoch: 1/100
Epoch: 001, Training: Loss: 1.5910, Accuracy: 28.1553%, 
		Validation: Loss: 1.4190, Accuracy: 27.5281%, Time: 1.2378s
Best Accuracy for validation : 0.2753 at epoch 001
Accuracy: 0.275281 classID:1 acc:0.775510  classID:2 acc:0.050000  classID:3 acc:0.000000  classID:4 acc:0.227273 
Epoch: 2/100
Epoch: 002, Training: Loss: 1.5390, Accuracy: 29.1262%, 
		Validation: Loss: 1.4028, Accuracy: 26.4045%, Time: 0.3904s
Best Accuracy for validation : 0.2753 at epoch 001
Accuracy: 0.264045 classID:1 acc:0.877551  classID:2 acc:0.000000  classID:3 acc:0.015385  classID:4 acc:0.068182 
Epoch: 3/100
Epoch: 003, Training: Loss: 1.4299, Accuracy: 32.0388%, 
		Validation: Loss: 1.3680, Accuracy: 31.4607%, Time: 0.3842s
Best Accuracy for validation : 0.3146 at epoch 003
Accuracy: 0.314607 classID:1 acc:0.938776  classID:2 acc:0.000000  classID:3 acc:0.138462  classID:4 acc:0.022727 
Epoch: 4/100
Epoch: 004, Training: Loss: 1.4794, Accuracy: 27.1845%, 
		Validation: Loss: 1.3622, Accuracy

KeyboardInterrupt: 

In [ ]:
history = np.array(history)
plt.plot(history[:, 0:2])
plt.legend(['Tr Loss', 'Val Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.ylim(0, 3)
plt.show()
 
plt.plot(history[:, 2:4])
plt.legend(['Tr Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.show()
